In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt

In [2]:
import tqdm
from SenseTheFlow import config
config.bar = tqdm.tqdm_notebook

from SenseTheFlow.model import Model, DataParser
from SenseTheFlow.dataset import Dataset

/opt/anaconda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


LMDB Loading won't be available


In [3]:
import mnist
import random
from scipy.ndimage.interpolation import rotate

def mnist_generator(images_fn, labels_fn):
    images, labels = images_fn(), labels_fn()
    
    for i in range(images.shape[0]):
        img, label = images[i, :, :, np.newaxis], labels[i]
        img = img.astype(np.float32)
        cls = random.randint(0, 1)
        
        if cls == 1:
            img = rotate(img, 15, reshape=False)
            
        yield (img, {'label': [int(label)], 'cls': [int(cls)]})

In [4]:
def label_fn(features, latent, labels, mode, params):
    features = tf.layers.conv2d(features, 32, 3, data_format=params['data_format'], activation=tf.nn.relu)
    features = tf.layers.conv2d(features, 64, 3, data_format=params['data_format'], activation=tf.nn.relu)
    features = tf.layers.max_pooling2d(features, 2, 1, data_format=params['data_format'])
    features = tf.layers.flatten(features)
    features = tf.layers.dense(features, params['latent_space_dimensions'], activation=tf.nn.relu)
    features = tf.concat((features, latent), axis=-1)
    features = tf.layers.dense(features, params['latent_space_dimensions'], activation=tf.nn.relu)
    logits = tf.layers.dense(features, params['num_classes'])
    
    # Final classification
    softmax = tf.nn.softmax(logits)
    cls = tf.argmax(softmax, axis=-1)
    
    # During inference we don't have labels
    loss = 0
    if labels is not None:
        loss = tf.nn.softmax_cross_entropy_with_logits(
            labels=tf.one_hot(labels, params['num_classes']),
            logits=logits
        )
    
    return cls, tf.reduce_mean(loss)

In [7]:
from crossgrad import *

def model_fn(features, labels, mode, params):
    with tf.device('/gpu:1'):
        predictions, loss = crossgrad(mode, label_fn, features, labels['cls'], labels['label'], 10, 10, 0.5, 0.5, params=params)
        
        accuracy_d1 = tf.metrics.accuracy(predictions['d1'], labels['cls'])
        accuracy_l1 = tf.metrics.accuracy(predictions['l1'], labels['label'])
        
        accuracy_d2 = tf.metrics.accuracy(predictions['d2'], labels['cls'])
        accuracy_l2 = tf.metrics.accuracy(predictions['l2'], labels['label'])        
        
        # Fetch global step
        global_step = tf.train.get_or_create_global_step()
        
        if params.get('summaries'):
            tf.summary.scalar('metrics/epoch/accuracy/domain/1', accuracy_d1[1])
            tf.summary.scalar('metrics/epoch/accuracy/labels/1', accuracy_l1[1])

            tf.summary.scalar('metrics/epoch/accuracy/domain/2', accuracy_d2[1])
            tf.summary.scalar('metrics/epoch/accuracy/labels/2', accuracy_l2[1])
            
            tf.summary.scalar('global_step/step', global_step)
        
        if mode == tf.estimator.ModeKeys.PREDICT:
            return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

        if mode == tf.estimator.ModeKeys.TRAIN:
            optimizer = tf.train.RMSPropOptimizer(learning_rate=0.001)

            # Batch norm requires update_ops to be added as a train_op dependency.
            update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
            with tf.control_dependencies(update_ops + [accuracy_d1[0], accuracy_l1[0], accuracy_d2[0], accuracy_l2[0]]):
                train_op = optimizer.minimize(loss, global_step)
        else:
            train_op = None

        return tf.estimator.EstimatorSpec(
            mode=mode,
            predictions=predictions,
            loss=loss,
            train_op=train_op,
            eval_metric_ops={
                'accuracy/domain/1': accuracy_d1, 
                'accuracy/labels/1': accuracy_l1,
                'accuracy/domain/2': accuracy_d2, 
                'accuracy/labels/2': accuracy_l2
            })

In [ ]:
params = {
    'data_format': None,
    'batch_size': 32,
    'num_domain': 2,
    'num_classes': 10,
    'summaries': True
}

model_dir = '/data/storage/deepglobe/models/{}'.format('crossgrad')

config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True)
config.gpu_options.allow_growth = True

tf.logging.set_verbosity(tf.logging.ERROR)

with Model(model_fn, model_dir, config=config, params=params, delete_existing=True) as model:
    data_parser = DataParser()
    data_parser.train_from_generator(
        generator=lambda: mnist_generator(mnist.train_images, mnist.train_labels),
        output_types=(tf.float32, {'label': tf.int32, 'cls': tf.int32}),
        output_shapes=([28, 28, 1], {'label': [1], 'cls': [1]}),
        batch_size=params['batch_size']
    )
    
    data_parser.eval_from_generator(
        generator=lambda: mnist_generator(mnist.test_images, mnist.test_labels),
        output_types=(tf.float32, {'label': tf.int32, 'cls': tf.int32}),
        output_shapes=([28, 28, 1], {'label': [1], 'cls': [1]}),
        batch_size=params['batch_size']
    )
    
    model.data(data_parser)
    
    model.train(100, 1, eval_summary=[
        'metrics/epoch/accuracy/domain/1', 'metrics/epoch/accuracy/domain/2',
        'metrics/epoch/accuracy/labels/1', 'metrics/epoch/accuracy/labels/2'
    ])
